#### Скачивание данных

In [6]:
!pip3 install vk
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Аккуратно скачаем все посты из группы.

* Используем tqdm для удобного вывода прогресса.
* На всякий случай запомним пост полностью, чтобы не скачивать все заного, если что-то потом "всплывет".
* Просмотры появились в постах не так давно, в старых их просто не будет. Предусмотрим это в коде.
* Вложений тоже может не быть, поэтому будем с ними аккуратнее.

In [21]:
import datetime
import pandas as pd

In [22]:
import vk
from tqdm import tqdm

session = vk.Session(access_token="38479e4a81d854219057927293934ed4d12b1c91afb966101385de82fc3d24310c0d5a039d4a907764504")
api = vk.API(session)

columns = ['text', 'views', 'likes', 'comments', 'is_ad', 'attachments', 'is_repost', 'time', 'raw']
df = pd.DataFrame(columns=columns)

result = api.wall.get(domain='tproger', count=1, v=5.103)
count = result['count']

for offset in tqdm(range(0, count, 100), position=0, leave=True):
    try:
        result = api.wall.get(domain='tproger', count=100, offset=offset, v=5.103)
        for post in result['items']:
            if "views" not in post:
                raise Exception("No views in post")
            """
            print('--- ' * 10)
            print("Пост №:", post['id'])
            print("Размер текста:", len(post['text']))
            print("Просмотры:", post['views']['count'])
            print("Лайки:", post['likes']['count'])
            print("Коменты:", post['comments']['count'])
            print("Реклама?", post['marked_as_ads'])
            print("Число вложений:", len(post['attachments']))
            print("Репост?", post.get('copy_history', None) != None)
            print("Время публикации:", post['date'])
            """
            attachments = post.get('attachments', [])

            df.loc[post['id']] = [
                post['text'],  post['views']['count'], post['likes']['count'], post['comments']['count'],
                post['marked_as_ads'], attachments, post.get('copy_history', None) != None, 
                post['date'], post
            ]
    except Exception as e:
        print("Error:", e)
        break
        


print(df.shape)
df.head()


 27%|██▋       | 67/244 [01:47<05:13,  1.77s/it]

Error: No views in post
(6777, 9)


,text,views,likes,comments,is_ad,attachments,is_repost,time,raw
1653786,Шарите в Java? Есть отличный шанс это продемон...,90955,416,156,0,"[{'type': 'photo', 'photo': {'id': 457676504, ...",False,1581689820,"{'id': 1653786, 'from_id': -30666517, 'owner_i..."
1655188,Генерация случайных чисел — задача не из прост...,8108,51,11,0,"[{'type': 'video', 'video': {'access_key': '0e...",False,1582476180,"{'id': 1655188, 'from_id': -30666517, 'owner_i..."
1655157,Git pull и git fetch — в чём разница?\n\nЧитай...,27263,126,16,0,"[{'type': 'photo', 'photo': {'id': 457676335, ...",False,1582456560,"{'id': 1655157, 'from_id': -30666517, 'owner_i..."
1655094,Начните своё утро с годного музыкального дипфе...,43855,165,20,0,"[{'type': 'video', 'video': {'access_key': '68...",False,1582438440,"{'id': 1655094, 'from_id': -30666517, 'owner_i..."
1655065,"В обучении программированию, как и в любом дру...",32126,166,7,0,"[{'type': 'link', 'link': {'url': 'https://tpr...",False,1582385580,"{'id': 1655065, 'from_id': -30666517, 'owner_i..."


#### Смотрим на данные и правим типы

Итак, мы скачали почти 7000 постов. Посмотрим на типы данных колонок.

In [ ]:
df = pd.read_csv("csv/tipical_proger.csv", index_col=0)

In [23]:
for column in columns:
    print(f'{column}: {df[column].dtype}, {len(df[column])}')

text: object, 6777
views: object, 6777
likes: object, 6777
comments: object, 6777
is_ad: object, 6777
attachments: object, 6777
is_repost: object, 6777
time: object, 6777
raw: object, 6777


1) Во всех колонках одинаковое число записей, пропущенных значений нет. Иначе мы смогли бы использовать `dropna` или `fillna`, чтобы их заполнить.

2) У все колонок тип object (категориальные данные), хотя по факту в некоторых числа. Попробуем преобразовать тип.

In [24]:
num_columns = ['views', 'likes', 'comments', 'is_ad', 'is_repost', 'time']

for column in num_columns:
    df[column] = df[column].astype(int)

In [25]:
for column in columns:
    print(f'{column}: {df[column].dtype}, {len(df[column])}')

text: object, 6777
views: int64, 6777
likes: int64, 6777
comments: int64, 6777
is_ad: int64, 6777
attachments: object, 6777
is_repost: int64, 6777
time: int64, 6777
raw: object, 6777


Отлично, теперь в колонках числа, можем на них посмотреть.

In [26]:
df.describe()

,views,likes,comments,is_ad,is_repost,time
count,6777.000000,6777.000000,6777.000000,6777.000000,6777.000000,6.777000e+03
mean,45118.466135,263.589641,23.133392,0.115243,0.039988,1.527585e+09
std,18686.296457,307.331127,69.118073,0.319338,0.195946,2.927529e+07
min,8108.000000,2.000000,0.000000,0.000000,0.000000,1.483218e+09
25%,34219.000000,66.000000,5.000000,0.000000,0.000000,1.500756e+09
50%,41679.000000,157.000000,13.000000,0.000000,0.000000,1.528286e+09
75%,49973.000000,343.000000,26.000000,0.000000,0.000000,1.552127e+09
max,287266.000000,5044.000000,4558.000000,1.000000,1.000000,1.582476e+09


In [27]:
import matplotlib.pyplot as plt
%matplotlib inline

num_columns = ['views', 'likes', 'comments', 'is_ad', 'is_repost', 'time']

for column in num_columns:
    print(column)
    df[column].hist()
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

В данных выше мы можем увидеть интересные моменты: есть посты с аномально большим числом лайков и комментов. Можно на них посмотреть.

In [28]:
df.sort_values('comments', ascending=False)[:5]

,text,views,likes,comments,is_ad,attachments,is_repost,time,raw
1587910,В приложении «ВКонтакте» для Android появилась...,122610,312,4558,0,"[{'type': 'photo', 'photo': {'id': 456322633, ...",0,1555400095,"{'id': 1587910, 'from_id': -30666517, 'owner_i..."
1611436,Давайте устроим пятничный интерактив.\n\nВ ком...,83106,119,1936,0,"[{'type': 'photo', 'photo': {'id': 457405135, ...",0,1563542940,"{'id': 1611436, 'from_id': -30666517, 'owner_i..."
1579542,Сегодня в России прошли митинги против изоляци...,79485,492,596,0,"[{'type': 'photo', 'photo': {'id': 456277864, ...",0,1552245134,"{'id': 1579542, 'from_id': -30666517, 'owner_i..."
1624064,"Ностальгируем, не стесняемся",59647,180,576,0,"[{'type': 'photo', 'photo': {'id': 457486163, ...",0,1568370266,"{'id': 1624064, 'from_id': -30666517, 'owner_i..."
1645176,Что вас ждёт в Новом году? Предлагаем вам загл...,43820,37,538,0,"[{'type': 'photo', 'photo': {'id': 457635228, ...",0,1577714813,"{'id': 1645176, 'from_id': -30666517, 'owner_i..."


In [29]:
df.sort_values('likes', ascending=False)[:5]

,text,views,likes,comments,is_ad,attachments,is_repost,time,raw
1566145,,212396,5044,144,0,"[{'type': 'photo', 'photo': {'id': 456272768, ...",0,1546290005,"{'id': 1566145, 'from_id': -30666517, 'owner_i..."
1495237,Видеокурс «Основы программирования». \n \nРусс...,64537,4359,11,0,"[{'type': 'video', 'video': {'access_key': '76...",0,1500307628,"{'id': 1495237, 'from_id': -30666517, 'owner_i..."
1452322,"Year++, программисты!",135397,2972,127,0,"[{'type': 'photo', 'photo': {'id': 456240507, ...",0,1483218155,"{'id': 1452322, 'from_id': -30666517, 'owner_i..."
1483520,Подготовили для вас перевод подробного руковод...,96807,2925,17,0,"[{'type': 'link', 'link': {'url': 'https://tpr...",0,1493226250,"{'id': 1483520, 'from_id': -30666517, 'owner_i..."
1520134,"Привет. Просто анимированная форма логина, зац...",76543,2661,66,0,"[{'type': 'doc', 'doc': {'id': 460704137, 'own...",0,1519661280,"{'id': 1520134, 'from_id': -30666517, 'owner_i..."


#### Feature Engineering и предварительная обработка

Визуально мы заметили, что некоторые посты отмечены хэш-тегом "\#партнерский". Попробуем найти такие посты и посмотреть, отмечены ли они как реклама.

Для это применим функцию к столбцу text и получим новый столбец.

In [30]:


def is_partner(text):
    if "#партнерский" in text.replace('ё', 'е').lower():
        return 1
    else:
        return 0
    
df['is_partner'] = df['text'].apply(is_partner)

In [31]:
df['is_partner'].hist()
plt.show()
df['is_ad'].hist()
plt.show()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

Интересно, а все ли посты, отмеченные, как партнерские, помечены, как реклама.

In [32]:
df[ df['is_partner'] == 1 ]['is_ad'].hist()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

Оказывается, нет! Выходит, мы только что придумали, как достать новую информацию из уже имеющихся данных. Это называется **feature ingineerring**. Именно в этом и состоит *искусство* машинного обучения.

https://habr.com/ru/company/mlclass/blog/248129/

Еще у нас есть время публикации. Попробуем достать из него, например, день недели.

https://stackoverflow.com/questions/9847213/how-do-i-get-the-day-of-week-given-a-date

In [33]:
from datetime import date
import calendar

# test
print("Today:", calendar.day_name[datetime.datetime.today().weekday()])


def get_weekday(unix_time):
    date = datetime.datetime.fromtimestamp(unix_time)
    return calendar.day_name[date.weekday()]


df['weekday'] = df['time'].apply(get_weekday)   

Today: Sunday


In [34]:
df['weekday'].hist()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

Как отправить эти данные в машину? Превратить в числа. Первое что приходит в голову - просто выдать номер для каждого для недели.

Но тогда получится, что воскренье больше понедельника, что не так - они идут подряд, а это может сбить некоторые модели с толку. Поэтому вместо этого создадим по булевому признаку для каждого дня недели.

In [35]:
df = pd.concat([df, pd.get_dummies(df['weekday'], prefix='is_')] , axis=1)

In [36]:
df.head()

,text,views,likes,comments,is_ad,attachments,is_repost,time,raw,is_partner,weekday,is__Friday,is__Monday,is__Saturday,is__Sunday,is__Thursday,is__Tuesday,is__Wednesday
1653786,Шарите в Java? Есть отличный шанс это продемон...,90955,416,156,0,"[{'type': 'photo', 'photo': {'id': 457676504, ...",0,1581689820,"{'id': 1653786, 'from_id': -30666517, 'owner_i...",0,Friday,1,0,0,0,0,0,0
1655188,Генерация случайных чисел — задача не из прост...,8108,51,11,0,"[{'type': 'video', 'video': {'access_key': '0e...",0,1582476180,"{'id': 1655188, 'from_id': -30666517, 'owner_i...",0,Sunday,0,0,0,1,0,0,0
1655157,Git pull и git fetch — в чём разница?\n\nЧитай...,27263,126,16,0,"[{'type': 'photo', 'photo': {'id': 457676335, ...",0,1582456560,"{'id': 1655157, 'from_id': -30666517, 'owner_i...",0,Sunday,0,0,0,1,0,0,0
1655094,Начните своё утро с годного музыкального дипфе...,43855,165,20,0,"[{'type': 'video', 'video': {'access_key': '68...",0,1582438440,"{'id': 1655094, 'from_id': -30666517, 'owner_i...",0,Sunday,0,0,0,1,0,0,0
1655065,"В обучении программированию, как и в любом дру...",32126,166,7,0,"[{'type': 'link', 'link': {'url': 'https://tpr...",0,1582385580,"{'id': 1655065, 'from_id': -30666517, 'owner_i...",0,Saturday,0,0,1,0,0,0,0


Сделаем еще пару признаков - длина текста и количество вложений.

In [37]:
df['attachments_len'] = df['attachments'].apply(lambda x:len(x))
df['text_len'] = df['text'].apply(lambda x:len(x))

#### Убираем аутлайнеров

Как мы видели ранее, в наших данных есть очень нетипичные почты, которые сильно отличаются от остальных. Например, пост про новый год, у которого очень много лайков.

Если мы хотим сделать модель с более высокой обобщающей способностью, такие данные полезно отбросить. Например, можно посмотреть на стандартное отклонение и отбросить все, что лежит сильно дальше среднего.

Однако, становится ли от этого лучше, стоит отдельно проверить позже.

In [38]:
learn_df = df.drop(columns=['text', 'attachments', 'raw', 'weekday'])

In [39]:
def not_outliner(value, mean_value, std_value):
    if mean_value - 3 * std_value < value < mean_value + 3 * std_value:
        return True
    return False

for column in learn_df.columns:
    mean_value = df[column].mean()
    std_value = df[column].std()
    regular_posts = learn_df[column].apply(lambda x: not_outliner(x, mean_value, std_value))
    learn_df = learn_df[regular_posts]

print(df.shape)
print(learn_df.shape)

(6777, 20)
(5708, 16)


#### Обучаем модели

In [40]:
X = learn_df.drop(columns=['views'])
Y = learn_df['views']

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

ModuleNotFoundError: No module named 'sklearn'

In [42]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

ModuleNotFoundError: No module named 'sklearn'

In [43]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

models = {
    "linear": LinearRegression(),
    "linear with regularisation": Ridge(),
    "tree": DecisionTreeRegressor(max_depth=5),
    "forest": RandomForestRegressor(),
    "gradient_boosting": GradientBoostingRegressor()
}

for model_name in models:
    models[model_name].fit(X_train, Y_train)
    train_predictions = models[model_name].predict(X_train)
    test_predictions = models[model_name].predict(X_test)
    
    print("")
    print("Model: ", model_name)
    print("Train MSE:", mean_squared_error(Y_train, train_predictions) ** 0.5)
    print("Test MSE:", mean_squared_error(Y_test, test_predictions) ** 0.5)

ModuleNotFoundError: No module named 'sklearn'

На данный момент лучше всего работает случайный лес и градиентный бустинг. Можно посмотреть, на какие признаки он опирается.

In [44]:
for (w, n) in sorted(zip(models["forest"].feature_importances_, X.columns)):
    print(n, " - ", w)

NameError: name 'models' is not defined

Чтобы посмотреть, влияют ли признаки негативно или положительно можем дополнительно посмотреть на регрессию.

In [45]:
for (w, n) in sorted(zip(models["linear"].coef_, X.columns)):
    print(n, " - ", w)

NameError: name 'models' is not defined

Выходит, что лучше всего публиковать посты в воскресенье, а хуже всего в среду. Много вложений - это плохо.